In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from IPython.core.display import HTML

# Combining all data

## Charges

In [2]:
charges_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_charges_20100101-20101231_TRAVIS.csv')
charges_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_charges_20110101-20111231_TRAVIS.csv')
charges_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_charges_20120101-20121231_TRAVIS.csv')
charges_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_charges_20130101-20131231_TRAVIS.csv')
charges_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_charges_20140101-20141231_TRAVIS.csv')
charges_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_charges_20150101-20151231_TRAVIS.csv')
charges_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_charges_20160101-20161231_TRAVIS.csv')
charges_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_charges_20170101-20170829_TRAVIS.csv')

In [3]:
charges = pd.concat([charges_10, charges_11, charges_12, charges_13, charges_14, charges_15, charges_16, charges_17])

### Checking no new duplicates from concat

In [4]:
# charges['dup'] = charges.Crash_ID.map(str) + charges.Unit_Nbr.map(str) + charges.Prsn_Nbr.map(str) + charges.Charge_Cat_ID.map(str) + charges.Charge + charges.Citation_Nbr.map(str)
# charges = charges.sort_values(by='dup')
# charges[charges.dup.duplicated(keep=False)]

## Crash

In [5]:
#Column 22 (Rpt_Block_Num) is almost all int except for when they put 111 1/2 or 111 BLK
rpt_block_num_cnvtr = lambda x: x.split(' ')[0] if len(x.split(' ')) > 1 and x.split(' ')[1] in ['1/2', 'BLK'] else x
rpt_sec_hwy_sfx_dict = {'W': 22, 'R': 17, 'E': 5, 'S': 18, 'N': 14, 'M': 13, 'A': 1}
rpt_sec_hwy_sfx_cnvtr = lambda x: rpt_sec_hwy_sfx_dict[x] if type(x) == str and len(x) > 0 else x
crash_10_cnvtr = {22: rpt_block_num_cnvtr}
crash_12_cnvtr = {22: rpt_block_num_cnvtr, 35: rpt_sec_hwy_sfx_cnvtr, 20: rpt_sec_hwy_sfx_cnvtr}
dtypes = {'Rpt_Sec_Hwy_Num': str, 'Rpt_Sec_Block_Num': str, 'Street_Nbr': str, \
          'Rpt_CrossingNumber': str, 'Rpt_CrossingNumber': str, 'CrossingNumber': str, 'RRCo': str}

In [6]:
# crash_10 skipped rows have data split incorrectly in csv file starting at col 33
# crash_15, 16, 17 skipped rows have Rpt_Hwy_Sfx values entered incorrectly 
#     (entered actual sfx rather than data dict code)
crash_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_crash_20100101-20101231_TRAVIS.csv', \
                       converters=crash_10_cnvtr, skiprows=[2495, 3258, 3580, 8252, 9523], dtype=dtypes)
crash_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_crash_20110101-20111231_TRAVIS.csv', \
                      converters=crash_10_cnvtr, dtype=dtypes)
crash_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_crash_20120101-20121231_TRAVIS.csv', \
                      converters=crash_12_cnvtr, dtype=dtypes)
crash_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_crash_20130101-20131231_TRAVIS.csv', \
                      dtype=dtypes)
crash_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_crash_20140101-20141231_TRAVIS.csv', \
                       dtype=dtypes)
crash_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_crash_20150101-20151231_TRAVIS.csv', \
                       converters=crash_12_cnvtr, dtype=dtypes, \
                       skiprows=[5177, 5446, 6653, 9839, 9841, 14381, 19176])
crash_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_crash_20160101-20161231_TRAVIS.csv', \
                       converters=crash_12_cnvtr, dtype=dtypes, skiprows=[2432, 3794, 3795, 8077, 8188, 20173])
crash_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_crash_20170101-20170829_TRAVIS.csv', \
                      converters=crash_12_cnvtr, dtype=dtypes, skiprows=[4480])

/home/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
crash = pd.concat([crash_10, crash_11, crash_12, crash_13, crash_14, crash_15, crash_16, crash_17])

### Checking no new duplicates from concat

In [8]:
# crash[crash.Crash_ID.duplicated()]

## Damages

In [9]:
damages_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_damages_20100101-20101231_TRAVIS.csv')
damages_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_damages_20110101-20111231_TRAVIS.csv')
damages_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_damages_20120101-20121231_TRAVIS.csv')
damages_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_damages_20130101-20131231_TRAVIS.csv')
damages_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_damages_20140101-20141231_TRAVIS.csv')
damages_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_damages_20150101-20151231_TRAVIS.csv')
damages_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_damages_20160101-20161231_TRAVIS.csv')
damages_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_damages_20170101-20170829_TRAVIS.csv')

In [10]:
damages = pd.concat([damages_10, damages_11, damages_12, damages_13, damages_14, damages_15, damages_16, damages_17])

### Checking no new duplicates from concat

In [11]:
# damages['dup'] = damages['Crash_ID'].map(str) + damages['Damaged_Property'].map(str)
# damages = damages.sort_values(by='dup')
# duplicates shown exist in csv. must be 'multiple counts of damages' or something like that
# damages[damages['dup'].duplicated(keep=False)]


## Endorsements

In [12]:
endorsements_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_endorsements_20100101-20101231_TRAVIS.csv')
endorsements_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_endorsements_20110101-20111231_TRAVIS.csv')
endorsements_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_endorsements_20120101-20121231_TRAVIS.csv')
endorsements_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_endorsements_20130101-20131231_TRAVIS.csv')
endorsements_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_endorsements_20140101-20141231_TRAVIS.csv')
endorsements_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_endorsements_20150101-20151231_TRAVIS.csv')
endorsements_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_endorsements_20160101-20161231_TRAVIS.csv')
endorsements_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_endorsements_20170101-20170829_TRAVIS.csv')

In [13]:
endorsements = pd.concat([endorsements_10, endorsements_11, endorsements_12, endorsements_13, endorsements_14, endorsements_15, endorsements_16, endorsements_17])

### Checking no new duplicates from concat

In [14]:
# endorsements['dup'] = endorsements['Crash_ID'].map(str) + endorsements['Unit_Nbr'].map(str) + endorsements['Drvr_Lic_Endors_ID'].map(str)
# endorsements = endorsements.sort_values(by='dup')
# endorsements[endorsements['dup'].duplicated(keep=False)]


## Person

In [15]:
person_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_person_20100101-20101231_TRAVIS.csv')
person_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_person_20110101-20111231_TRAVIS.csv')
person_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_person_20120101-20121231_TRAVIS.csv')
person_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_person_20130101-20131231_TRAVIS.csv')
person_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_person_20140101-20141231_TRAVIS.csv')
person_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_person_20150101-20151231_TRAVIS.csv')
person_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_person_20160101-20161231_TRAVIS.csv')
person_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_person_20170101-20170829_TRAVIS.csv')

In [16]:
person = pd.concat([person_10, person_11, person_12, person_13, person_14, person_15, person_16, person_17])

### Checking no new duplicates from concat

In [17]:
# person['dup'] = person['Crash_ID'].map(str) + person['Unit_Nbr'].map(str) + person['Prsn_Nbr'].map(str)
# person = person.sort_values(by='dup')
# person[person['dup'].duplicated(keep=False)]


## Unit

In [18]:
cmv_carrier_zip_cnvtr = lambda x: x.replace('-', '') if '-' in str(x) else x
unit_10_cnvtr = {31: cmv_carrier_zip_cnvtr}
dtypes = {80: str}

In [19]:
# Unit_16 skipped rows had column values shifted in the csv
unit_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_unit_20100101-20101231_TRAVIS.csv', \
                     converters=unit_10_cnvtr)
unit_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_unit_20110101-20111231_TRAVIS.csv')
unit_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_unit_20120101-20121231_TRAVIS.csv')
unit_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_unit_20130101-20131231_TRAVIS.csv')
unit_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_unit_20140101-20141231_TRAVIS.csv')
unit_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_unit_20150101-20151231_TRAVIS.csv')
unit_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_unit_20160101-20161231_TRAVIS.csv', \
                     skiprows=[43205], dtype=dtypes)
unit_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_unit_20170101-20170829_TRAVIS.csv')

/home/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
unit = pd.concat([unit_10, unit_11, unit_12, unit_13, unit_14, unit_15, unit_16, unit_17])

### Checking no new duplicates from concat

In [21]:
# unit['dup'] = unit['Crash_ID'].map(str) + unit['Unit_Nbr'].map(str)
# unit = unit.sort_values(by='dup')
# unit[unit['dup'].duplicated(keep=False)]


## Restrictions

In [22]:
restrictions_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_restrictions_20100101-20101231_TRAVIS.csv')
restrictions_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_restrictions_20110101-20111231_TRAVIS.csv')
restrictions_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_restrictions_20120101-20121231_TRAVIS.csv')
restrictions_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_restrictions_20130101-20131231_TRAVIS.csv')
restrictions_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_restrictions_20140101-20141231_TRAVIS.csv')
restrictions_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_restrictions_20150101-20151231_TRAVIS.csv')
restrictions_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_restrictions_20160101-20161231_TRAVIS.csv')
restrictions_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_restrictions_20170101-20170829_TRAVIS.csv')

In [23]:
restrictions = pd.concat([restrictions_10, restrictions_11, restrictions_12, restrictions_13, restrictions_14, restrictions_15, restrictions_16, restrictions_17])

### Checking no new duplicates from concat

In [24]:
# restrictions['dup'] = restrictions['Crash_ID'].map(str) + restrictions['Unit_Nbr'].map(str) + restrictions['Drvr_Lic_Restric_ID'].map(str)
# restrictions = restrictions.sort_values(by='dup')
# restrictions[restrictions['dup'].duplicated(keep=False)]


## PrimaryPerson

In [25]:
primaryperson_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_primaryperson_20100101-20101231_TRAVIS.csv')
primaryperson_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_primaryperson_20110101-20111231_TRAVIS.csv')
primaryperson_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_primaryperson_20120101-20121231_TRAVIS.csv')
primaryperson_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_primaryperson_20130101-20131231_TRAVIS.csv')
primaryperson_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_primaryperson_20140101-20141231_TRAVIS.csv')
primaryperson_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_primaryperson_20150101-20151231_TRAVIS.csv')
primaryperson_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_primaryperson_20160101-20161231_TRAVIS.csv')
primaryperson_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_primaryperson_20170101-20170829_TRAVIS.csv')

In [26]:
primaryperson = pd.concat([primaryperson_10, primaryperson_11, primaryperson_12, primaryperson_13, primaryperson_14, primaryperson_15, primaryperson_16, primaryperson_17])

### Checking no new duplicates from concat

In [27]:
# primaryperson['dup'] = primaryperson['Crash_ID'].map(str) + primaryperson['Unit_Nbr'].map(str) + primaryperson['Prsn_Nbr'].map(str)
# primaryperson = primaryperson.sort_values(by='dup')
# primaryperson[primaryperson['dup'].duplicated(keep=False)]

## Joining each section of data

In [28]:
# combining person tables since almost identical, no more duplicate person columns
join = pd.merge(charges, crash, how='outer', on='Crash_ID')
join = pd.merge(join, damages, how='outer', on='Crash_ID')
join = pd.merge(join, endorsements, how='outer', on=['Crash_ID', 'Unit_Nbr'])
join = pd.merge(join, person, how='outer', on=['Crash_ID', 'Prsn_Nbr', 'Unit_Nbr'])
join = pd.merge(join, primaryperson, how='outer', on=['Crash_ID', 'Prsn_Nbr', 'Unit_Nbr'])
join = pd.merge(join, restrictions, how='outer', on=['Crash_ID', 'Unit_Nbr'])
join = pd.merge(join, unit, how='outer', on=['Crash_ID', 'Unit_Nbr'])

In [29]:
mini_join = join.sample(n=100)
HTML(mini_join.to_html())

,Crash_ID,Unit_Nbr,Prsn_Nbr,Charge_Cat_ID,Charge,Citation_Nbr,Crash_Fatal_Fl,Cmv_Involv_Fl,Schl_Bus_Fl,Rr_Relat_Fl,Medical_Advisory_Fl,Amend_Supp_Fl,Active_School_Zone_Fl,Crash_Date,Crash_Time,Case_ID,Local_Use,Rpt_CRIS_Cnty_ID,Rpt_City_ID,Rpt_Outside_City_Limit_Fl,Thousand_Damage_Fl,Rpt_Latitude,Rpt_Longitude,Rpt_Rdwy_Sys_ID,Rpt_Hwy_Num,Rpt_Hwy_Sfx,Rpt_Road_Part_ID,Rpt_Block_Num,Rpt_Street_Pfx,Rpt_Street_Name,Rpt_Street_Sfx,Private_Dr_Fl,Toll_Road_Fl,Crash_Speed_Limit,Road_Constr_Zone_Fl,Road_Constr_Zone_Wrkr_Fl,Rpt_Street_Desc,At_Intrsct_Fl,Rpt_Sec_Rdwy_Sys_ID,Rpt_Sec_Hwy_Num,Rpt_Sec_Hwy_Sfx,Rpt_Sec_Road_Part_ID,Rpt_Sec_Block_Num,Rpt_Sec_Street_Pfx,Rpt_Sec_Street_Name,Rpt_Sec_Street_Sfx,Rpt_Ref_Mark_Offset_Amt,Rpt_Ref_Mark_Dist_Uom,Rpt_Ref_Mark_Dir,Rpt_Ref_Mark_Nbr,Rpt_Sec_Street_Desc,Rpt_CrossingNumber,Wthr_Cond_ID,Light_Cond_ID,Entr_Road_ID,Road_Type_ID,Road_Algn_ID,Surf_Cond_ID,Traffic_Cntl_ID,Investigat_Notify_Time,Investigat_Notify_Meth,Investigat_Arrv_Time,Report_Date,Investigat_Comp_Fl,ORI_Number,Investigat_Agency_ID,Investigat_Area_ID,Investigat_District_ID,Investigat_Region_ID,Bridge_Detail_ID,Harm_Evnt_ID,Intrsct_Relat_ID,FHE_Collsn_ID,Obj_Struck_ID,Othr_Factr_ID,Road_Part_Adj_ID,Road_Cls_ID,Road_Relat_ID,Phys_Featr_1_ID,Phys_Featr_2_ID,Cnty_ID,City_ID,Latitude,Longitude,Hwy_Sys,Hwy_Nbr,Hwy_Sfx,Dfo,Street_Name,Street_Nbr,Control,Section,Milepoint,Ref_Mark_Nbr,Ref_Mark_Displ,Hwy_Sys_2,Hwy_Nbr_2,Hwy_Sfx_2,Street_Name_2,Street_Nbr_2,Control_2,Section_2,Milepoint_2,Txdot_Rptable_Fl,Onsys_Fl,Rural_Fl,Crash_Sev_ID,Pop_Group_ID,Located_Fl,Day_of_Week,Hwy_Dsgn_Lane_ID,Hwy_Dsgn_Hrt_ID,Hp_Shldr_Left,Hp_Shldr_Right,Hp_Median_Width,Base_Type_ID,Nbr_Of_Lane,Row_Width_Usual,Roadbed_Width,Surf_Width,Surf_Type_ID,Curb_Type_Left_ID,Curb_Type_Right_ID,Shldr_Type_Left_ID,Shldr_Width_Left,Shldr_Use_Left_ID,Shldr_Type_Right_ID,Shldr_Width_Right,Shldr_Use_Right_ID,Median_Type_ID,Median_Width,Rural_Urban_Type_ID,Func_Sys_ID,Adt_Curnt_Amt,Adt_Curnt_Year,Adt_Adj_Curnt_Amt,Pct_Single_Trk_Adt,Pct_Combo_Trk_Adt,Trk_Aadt_Pct,Curve_Type_ID,Curve_Lngth,Cd_Degr,Delta_Left_Right_ID,Dd_Degr,Feature_Crossed,Structure_Number,I_R_Min_Vert_Clear,Approach_Width,Bridge_Median_ID,Bridge_Loading_Type_ID,Bridge_Loading_In_1000_Lbs,Bridge_Srvc_Type_On_ID,Bridge_Srvc_Type_Under_ID,Culvert_Type_ID,Roadway_Width,Deck_Width,Bridge_Dir_Of_Traffic_ID,Bridge_Rte_Struct_Func_ID,Bridge_IR_Struct_Func_ID,CrossingNumber,RRCo,Poscrossing_ID,WDCode_ID,Standstop,Yield,Incap_Injry_Cnt_x,Nonincap_Injry_Cnt_x,Poss_Injry_Cnt_x,Non_Injry_Cnt_x,Unkn_Injry_Cnt_x,Tot_Injry_Cnt_x,Death_Cnt_x,MPO_ID,Investigat_Service_ID,Investigat_DA_ID,Investigator_Narrative,Damaged_Property,Drvr_Lic_Endors_ID,Prsn_Type_ID_x,Prsn_Occpnt_Pos_ID_x,Prsn_Injry_Sev_ID_x,Prsn_Age_x,Prsn_Ethnicity_ID_x,Prsn_Gndr_ID_x,Prsn_Ejct_ID_x,Prsn_Rest_ID_x,Prsn_Airbag_ID_x,Prsn_Helmet_ID_x,Prsn_Sol_Fl_x,Prsn_Alc_Spec_Type_ID_x,Prsn_Alc_Rslt_ID_x,Prsn_Bac_Test_Rslt_x,Prsn_Drg_Spec_Type_ID_x,Prsn_Drg_Rslt_ID_x,Prsn_Death_Time_x,Incap_Injry_Cnt_y,Nonincap_Injry_Cnt_y,Poss_Injry_Cnt_y,Non_Injry_Cnt_y,Unkn_Injry_Cnt_y,Tot_Injry_Cnt_y,Death_Cnt_y,Prsn_Type_ID_y,Prsn_Occpnt_Pos_ID_y,Prsn_Injry_Sev_ID_y,Prsn_Age_y,Prsn_Ethnicity_ID_y,Prsn_Gndr_ID_y,Prsn_Ejct_ID_y,Prsn_Rest_ID_y,Prsn_Airbag_ID_y,Prsn_Helmet_ID_y,Prsn_Sol_Fl_y,Prsn_Alc_Spec_Type_ID_y,Prsn_Alc_Rslt_ID_y,Prsn_Bac_Test_Rslt_y,Prsn_Drg_Spec_Type_ID_y,Prsn_Drg_Rslt_ID_y,Drvr_Drg_Cat_1_ID,Prsn_Death_Time_y,Incap_Injry_Cnt_x,Nonincap_Injry_Cnt_x,Poss_Injry_Cnt_x,Non_Injry_Cnt_x,Unkn_Injry_Cnt_x,Tot_Injry_Cnt_x,Death_Cnt_x,Drvr_Lic_Type_ID,Drvr_Lic_State_ID,Drvr_Lic_Cls_ID,Drvr_Zip,Drvr_Lic_Restric_ID,Unit_Desc_ID,Veh_Parked_Fl,Veh_HNR_Fl,Veh_Lic_State_ID,VIN,Veh_Mod_Year,Veh_Color_ID,Veh_Make_ID,Veh_Mod_ID,Veh_Body_Styl_ID,Emer_Respndr_Fl,Ownr_Zip,Fin_Resp_Proof_ID,Fin_Resp_Type_ID,Veh_Dmag_Area_1_ID,Veh_Dmag_Scl_1_ID,Force_Dir_1_ID,Veh_Dmag_Area_2_ID,Veh_Dmag_Scl_2_ID,Force_Dir_2_ID,Veh_Inventoried_Fl,Veh_Transp_Name,Veh_Transp_Dest,Veh_Cmv_Fl,Cmv_Fiveton_Fl,Cmv

## Merging complete, finding patterns

In [31]:
join[['Crash_Sev_ID', 'Crash_ID']].groupby('Crash_Sev_ID').agg('count')
# injry_sevr_map = {0: 'unknown', 1: 'incapacitating', 2: 'non-incapacitating', 3: 'possible', 4: 'killed', 5: 'none'}

,Crash_ID
Crash_Sev_ID,
0.0,5405
1.0,5355
2.0,32877
3.0,34307
4.0,865
5.0,85090


In [38]:
join[join['Crash_Sev_ID'] == 4].loc[:,['Crash_Date', 'Crash_Time', 'Rpt_Latitude', 'Rpt_Longitude']]

,Crash_Date,Crash_Time,Rpt_Latitude,Rpt_Longitude
1844,03/04/2010,07:36 PM,NaN,NaN
1845,03/04/2010,07:36 PM,NaN,NaN
8589,06/13/2010,12:01 AM,30.33300,-97.60170
8590,06/13/2010,12:01 AM,30.33300,-97.60170
8843,05/21/2010,01:25 PM,30.08917,-97.71253
8844,05/21/2010,01:25 PM,30.08917,-97.71253
10425,10/23/2010,02:20 AM,NaN,NaN
10426,10/23/2010,02:20 AM,NaN,NaN
11253,06/15/2010,09:36 PM,NaN,NaN
11254,06/15/2010,09:36 PM,NaN,NaN


### Seem interesting:
- Charge
- Crash_Sev_ID
- Investigate_Notify_Time
- Investigate_Notify_Meth
- Investigate_Arrv_Time

In [ ]:
for i in crash.columns:
    print i

In [ ]:
injry_sevr_2017 = crash.groupby('Crash_Sev_ID').agg('count')
injry_sevr_map = {0: 'unknown', 1: 'incapacitating', 2: 'non-incapacitating', 3: 'possible', 4: 'killed', 5: 'none'}

In [ ]:
injry_sevr_2017.index = injry_sevr_2017.index.map(lambda x: injry_sevr_map[x])

In [ ]:
injry_sevr_2017

What were the crashes that hurt/killed people like? What time of day, and where

In [ ]:
violent_2017 = join[(join['Crash_Sev_ID'] == 1) | (join['Crash_Sev_ID'] == 4)]
violent_2017 = violent_2017.loc[:, ('Day_of_Week', 'Crash_Date', 'Crash_Time', 'Rpt_Road_Part_ID', 'Rpt_Street_Name')]

In [ ]:
violent_2017.head()

In [ ]:
violent_2017.groupby('Day_of_Week').agg('count')

In [ ]:
violent_date_agg_2017 = violent_2017.groupby('Crash_Date').agg('count')
pd.to_datetime(violent_date_agg_2017.index)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(violent_date_agg_2017.index.to_datetime().tolist(), violent_date_agg_2017['Day_of_Week'])
plt.show()

In [ ]:
violent_weekday_agg_2017 = violent_2017.groupby('Day_of_Week').agg('count')
